In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import urllib.request

data_path = 'cullpdb_pc30_res3.0_R1.0_d191017_chains18877.gz'

# This just opens the data set and prints out the contents line by line
pdbIds = []

with open(data_path, 'r') as f:
    lines = f.read()

i = 0
    
for line in lines.split('\n'):
    if line[0:3] != 'IDs':
        pdbIds.append(line[0:5])
        i+=1
        if i==100:
            break
    
print(pdbIds)

['12ASA', '16VPA', '19HCA', '1A0AA', '1A0IA', '1A0TP', '1A1XA', '1A2PA', '1A2XB', '1A3AA', '1A41A', '1A5TA', '1A62A', '1A73A', '1A7JA', '1A8LA', '1A92A', '1A9XA', '1A9XB', '1AB8A', '1ABAA', '1AC5A', '1ACFA', '1AE9A', '1AEPA', '1AGJA', '1AH7A', '1AHOA', '1AHSA', '1AISB', '1AJOA', '1AKOA', '1AL3A', '1ALUA', '1ALYA', '1AM7A', '1AOCA', '1AOHA', '1AOLA', '1AOZA', '1APYB', '1AQZA', '1AT0A', '1ATGA', '1ATZA', '1AUOA', '1AVGI', '1AVOA', '1AVOB', '1AX8A', '1AXIB', '1AYOA', '1AZOA', '1AZSA', '1B0NB', '1B25A', '1B33N', '1B35A', '1B35B', '1B35C', '1B35D', '1B3UA', '1B5EA', '1B5QA', '1B63A', '1B77A', '1B89A', '1B8KA', '1B9HA', '1B9LA', '1B9MA', '1B9WA', '1BAZA', '1BB1A', '1BB9A', '1BCOA', '1BCPA', '1BCPB', '1BCPD', '1BCPF', '1BD8A', '1BDFA', '1BEAA', '1BF2A', '1BG1A', '1BG6A', '1BGFA', '1BH9A', '1BH9B', '1BHEA', '1BIAA', '1BJAA', '1BKRA', '1BKVA', '1BL0A', '1BM8A', '1BM9A', '1BMTA', '1BOOA', '1BOUA']


1. Find the popularity of different kind of helices: read the data set and analyze it to find the
popularity of each type of helices. 

In [2]:
from collections import Counter

helixList = []

for id in pdbIds:
    pdbFile = id[0:4]
    chain = id[-1].upper()
    url = 'https://files.rcsb.org/download/'+pdbFile+'.pdb'
    fileName = pdbFile + '.pdb'
    webFile = urllib.request.urlretrieve(url,fileName)
    pdbFile = open(fileName, "r")
    fileContents = pdbFile.readlines()
    for line in fileContents:
        if line[:5] == 'HELIX':
            helixList.append(line[38:40].replace(" ", ""))
    listCount = Counter(helixList)

print(listCount)

KeyboardInterrupt: 

2. Analyze the amino acids that prefer to form/be part of helices. Does an AA prefer to
form/be part of specific type of helices?

In [5]:
HelixLowerList = []
HelixUpperList = []
HelixTypeList = []
HelixAAList = []
AANumberList = []

for id in pdbIds:
    pdbFile = id[0:4]
    chain = id[-1].upper()
    url = 'https://files.rcsb.org/download/'+pdbFile+'.pdb'
    fileName = pdbFile + '.pdb'
    webFile = urllib.request.urlretrieve(url,fileName)
    pdbFile = open(fileName, "r")
    fileContents = pdbFile.readlines()
    for line in fileContents:
        if line[19:20] == chain:
            if line[:5] == 'HELIX':
                HelixLowerList.append(int(line[22:25].replace(" ", "")))
                HelixUpperList.append(int(line[34:37].replace(" ", "")))
                HelixTypeList.append(line[38:40].replace(" ", ""))
                df1 = pd.DataFrame(HelixLowerList, columns = ["HelixLowerBounds"])
                df1["HelixUpperBounds"] = HelixUpperList
                df1["HelixType"] = HelixTypeList
    for line in fileContents:
        if line[21:22] == chain:
            if line[:4] == 'ATOM':
                if line[13:16].replace(" ", "") == 'N':
                    for i in range(len(HelixLowerList)):
                        if ((int(line[23:26]) >= df1.HelixLowerBounds[i]) & (int(line[23:26]) <= df1.HelixUpperBounds[i])):
                            HelixAAList.append(line[17:20].replace(" ", ""))
                            AANumberList.append(int(line[23:26]))
                            #pd.set_option('display.max_rows', None)
                            df2 = pd.DataFrame(HelixAAList, columns = ["Helix Amino Acids"])
                            df2["AANumber"] = AANumberList
                            df2.append(pd.Series(name='HelixType'))
                            for j in range(len(AANumberList)):
                                df2["HelixType"][j] = HelixTypeList[i]
                                print(id)
                                display(df2)

KeyError: 'HelixType'

3. Analyze the length of helices (in terms of number of AAs). Is there any relation between
the type of a helix and its length?

In [6]:
helixDict1 = {}
helixDict2 = {}
helixList = []

for id in pdbIds:
    pdbFile = id[0:4]
    chain = id[-1].upper()
    url = 'https://files.rcsb.org/download/'+pdbFile+'.pdb'
    fileName = pdbFile + '.pdb'
    webFile = urllib.request.urlretrieve(url,fileName)
    pdbFile = open(fileName, "r")
    fileContents = pdbFile.readlines()
    for line in fileContents:
        if line[19:20] == chain:
            if line[:5] == 'HELIX':
                helixList.append(line[38:40].replace(" ", ""))
                helixNumber = int(line[8:10].replace(" ", ""))
                helixDict1["Helix type"] = line[38:40].replace(" ", "")
                lowerBound = int(line[22:25].replace(" ", ""))
                upperBound = int(line[34:37].replace(" ", ""))
                helixDict1["Helix length"] = float(upperBound - lowerBound)
                helixDict2[helixNumber] = dict(helixDict1)
    df3 = pd.DataFrame.from_dict(helixDict2, orient="index")
    print("\n Protein ID:", id)
    print(df3, "\n")
    typeMeans = df3.groupby("Helix type").aggregate(np.mean)
    print(typeMeans)


 Protein ID: 12ASA
   Helix type  Helix length
1           1          22.0
2           1           9.0
3           1          24.0
4           1           6.0
5           1          11.0
6           1          10.0
7           5           4.0
8           1           6.0
9           1           8.0
10          5           2.0
11          1           5.0 

            Helix length
Helix type              
1              11.222222
5               3.000000

 Protein ID: 16VPA
   Helix type  Helix length
1           1          11.0
2           1           9.0
3           5           3.0
4           5           2.0
5           1           9.0
6           5          54.0
7           1          44.0
8           1           3.0
9           1           3.0
10          1          13.0
11          1          12.0 

            Helix length
Helix type              
1              13.000000
5              19.666667

 Protein ID: 19HCA
   Helix type  Helix length
1           1           6.0
2       

KeyboardInterrupt: 